In [1]:
import cv2

In [2]:
import numpy as np
import time

In [3]:
# from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
# from tensorflow.keras.preprocessing.image import img_to_array
# from tensorflow.keras.models import load_model
# from imutils.video import VideoStream
# # import numpy as np
# import imutils
# import time
# # from cv2 import cv2
# import os

In [4]:
#Load YOLO
net = cv2.dnn.readNet("yolov3.weights","yolov3.cfg") # Original yolov3
#net = cv2.dnn.readNet("yolov3-tiny.weights","yolov3-tiny.cfg") #Tiny Yolo
classes = []
with open("coco.names","r") as f:
    classes = [line.strip() for line in f.readlines()]

In [5]:
print(classes)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [6]:
layer_names = net.getLayerNames()
outputlayers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [7]:
colors= np.random.uniform(0,255,size=(len(classes),3))

## Whole code

### Face Detetction

### Mask Detection

In [8]:
# maskNet = load_model("mask_detector.model")

In [9]:
# def detect_and_predict_mask(faces, maskNet):
# 	# grab the dimensions of the frame and then construct a blob
# 	# from it
# 	preds = []
# # 	frame = cv2.resize(face, (224, 224))

# 	if len(faces) > 0:

# 		faces = np.array(faces, dtype="float32")
# 		preds = maskNet.predict(faces, batch_size=32)

# 	# return a 2-tuple of the face locations and their corresponding
# 	# locations
# 	return preds
# def mask(frame,locs,preds):
# 	# face mask or not


# 	# loop over the detected face locations and their corresponding
# 	# locations
# 	for (box, pred) in zip(locs, preds):
# 		# unpack the bounding box and predictions
# 		(startX, startY, endX, endY) = box
# 		(mask, withoutMask) = pred

# 		# determine the class label and color we'll use to draw
# 		# the bounding box and text
# 		label = "Mask" if mask > withoutMask else "No Mask"
# 		color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

# 		# include the probability in the label
# 		label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

# 		# display the label and bounding box rectangle on the output
# 		# frame
# 		cv2.putText(frame, label, (startX, startY - 10),
# 		cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
# 		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
# 	return frame


In [10]:
face_Cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt2.xml')
# face_Cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt2.xml')
# cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
# face_Cascade=face_cascade.load('haarcascade_frontalface_default.xml')
# face_Cascade = cv2.CascadeClassifier('/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/cv2/data/haarcascade/haarcascade_frontalface_alt2.xml')

def Resize(image , x, y, h, w):
    return image[y:y+h, x:x+w]


def face(image):

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_Cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(60, 60),
        flags=cv2.CASCADE_SCALE_IMAGE
    )
#here if len(faces)>0 then check for mask
#     images=[]
#     locs=[]
    for (x, y, w, h) in faces:
        cv2.rectangle(image,(x,y),(x+w,y+h),colors[0],2)
#         cv2.putText(image,"Face",2)
        cv2.putText(image,"Face",(x,y+15),font,1,(255,255,255),2)
#         images.append(Resize(image,x,y,h,w))
#         locs.append((x,y,x+w,y+h))
#     preds=detect_and_predict_mask(images,maskNet)
#     image=mask(image,locs,preds)
# #         image = Resize(image, x, y, h, w )
    
    return image 

In [11]:
#loading image
cap=cv2.VideoCapture(0) #0 for 1st webcam
font = cv2.FONT_HERSHEY_PLAIN
starting_time= time.time()
frame_id = 0

while True:
    _,frame= cap.read() # 
    frame_id+=1
    
    height,width,channels = frame.shape
    count_humans=0
    count_faces=0
    count_mask=0
    #detecting objects
    blob = cv2.dnn.blobFromImage(frame,0.00392,(320,320),(0,0,0),True,crop=False) #reduce 416 to 320    

        
    net.setInput(blob)
    outs = net.forward(outputlayers)
    #print(outs[1])


    #Showing info on screen/ get confidence score of algorithm in detecting an object in blob
    class_ids=[]
    human_ids=[]
    face_ids=[]
    confidences=[]
    boxes=[]
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.3:
                #onject detected
                center_x= int(detection[0]*width)
                center_y= int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)

                #cv2.circle(img,(center_x,center_y),10,(0,255,0),2)
                #rectangle co-ordinaters
                x=int(center_x - w/2)
                y=int(center_y - h/2)
                #cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)

                boxes.append([x,y,w,h]) #put all rectangle areas
                confidences.append(float(confidence)) #how confidence was that object detected and show that percentage
                class_ids.append(class_id) #name of the object tha was detected

    indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.4,0.6)

    for i in range(len(boxes)):
        if i in indexes:
            x,y,w,h = boxes[i]
            label = str(classes[class_ids[i]])
            if(label=="person"):
                count_humans=count_humans+1
                human_ids.append(i)
                #this is for bounding box
                color = colors[class_ids[i]]
                cv2.rectangle(frame,(x,y),(x+w,y+h),color,2)
#                 frame=Resize(frame,x,y,w,h)
                confidence= confidences[i]
                cv2.putText(frame,label+" "+str(round(confidence,2)),(x,y+30),font,1,(255,255,255),2)
            


    elapsed_time = time.time() - starting_time
    fps=frame_id/elapsed_time
    cv2.putText(frame,"FPS:"+str(round(fps,2)),(10,50),font,2,(0,0,0),1)
    if(count_humans>0):
        #for each sec we are calling face if there exists any person in the camera view.
        print("yes at "+ str(elapsed_time))
        for i in range(len(human_ids)):
            frame = face(frame)
    else:
        print("no at "+ str(elapsed_time))
    
    cv2.imshow("Image",frame)
    key = cv2.waitKey(1) #wait 1ms the loop will start again and we will process the next frame
    
    if key == 27: #esc key stops the process
        break;
    
cap.release()    
cv2.destroyAllWindows()    
    
    

yes at 1.9296112060546875
yes at 2.9129798412323
yes at 3.917294502258301
yes at 4.914928197860718
yes at 6.000026226043701
yes at 6.967440366744995
yes at 7.957792520523071
yes at 8.814502477645874
yes at 9.752177476882935
yes at 10.70961880683899
yes at 11.572313785552979
yes at 12.453956127166748
yes at 13.290718793869019
yes at 14.164382219314575
yes at 15.024085283279419
yes at 15.874554872512817
yes at 16.743433713912964
yes at 17.589172840118408
yes at 18.43907642364502
yes at 19.310744047164917
yes at 20.154487133026123
yes at 21.0421142578125
yes at 21.895831823349
yes at 22.74556040763855
yes at 23.618227243423462
yes at 24.484912633895874
yes at 25.381514072418213
yes at 26.237225770950317
yes at 27.086955070495605
yes at 27.985553979873657
yes at 28.940998077392578
yes at 29.844032287597656
yes at 30.714704990386963
yes at 31.603330850601196
yes at 32.480984926223755
yes at 33.33769345283508
yes at 34.25244736671448
yes at 35.08920979499817
yes at 35.943923234939575
yes at 

In [12]:
# import numpy as np
# import cv2

# face_Cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt2.xml')

# def Resize(image , x, y, h, w):
#     return image[y:y+h, x:x+w]


# def face(image):

#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     faces = face_Cascade.detectMultiScale(
#         gray,
#         scaleFactor=1.1,
#         minNeighbors=5,
#         minSize=(60, 60),
#         flags=cv2.CASCADE_SCALE_IMAGE
#     )

#     for (x, y, w, h) in faces:
#         image = Resize(image, x, y, h, w )
    
#     return image 


# def Main():
#     Videocap = cv2.VideoCapture(0)

#     while(True):
#         ret, frame = Videocap.read()

#         frame = face(frame)

#         cv2.imshow('frame',frame)
#         #press 'q' to end web cam.  
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break

#     Videocap.release()
#     cv2.destroyAllWindows()


# Main()

In [13]:

            
# import numpy as np
# import cv2

# face_Cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt2.xml')

# def Resize(image , x, y, h, w):
#     return image[y:y+h, x:x+w]


# def face(image):

#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     faces = face_Cascade.detectMultiScale(
#         gray,
#         scaleFactor=1.1,
#         minNeighbors=5,
#         minSize=(60, 60),
#         flags=cv2.CASCADE_SCALE_IMAGE
#     )

#     for (x, y, w, h) in faces:
#         cv2.rectangle(image,(x,y),(x+w,y+h),colors[0],2)
    
#     return image 


# def Main():
#     Videocap = cv2.VideoCapture(0)

#     while(True):
#         ret, frame = Videocap.read()

#         frame = face(frame)

#         cv2.imshow('frame',frame)
#         #press 'q' to end web cam.  
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break

#     Videocap.release()
#     cv2.destroyAllWindows()


# Main()

In [14]:
# # import the necessary packages
# from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
# from tensorflow.keras.preprocessing.image import img_to_array
# from tensorflow.keras.models import load_model
# from imutils.video import VideoStream
# import numpy as np
# import imutils
# import time
# from cv2 import cv2
# import os

# def detect_and_predict_mask(frame, faces, maskNet):
# 	# grab the dimensions of the frame and then construct a blob
# 	# from it
# 	(h, w) = frame.shape[:2]
# # 	blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),
# # 		(104.0, 177.0, 123.0))

# 	# pass the blob through the network and obtain the face detections
# # 	faceNet.setInput(blob)
# # 	detections = faceNet.forward()
# # 	print(detections.shape)

# 	# initialize our list of faces, their corresponding locations,
# 	# and the list of predictions from our face mask network
# 	faces = []
# 	locs = []
# 	preds = []

# 	# loop over the detections
# # 	for i in range(0, detections.shape[2]):
# # 		# extract the confidence (i.e., probability) associated with
# # 		# the detection
# # 		confidence = detections[0, 0, i, 2]

# # 		# filter out weak detections by ensuring the confidence is
# # 		# greater than the minimum confidence
# # 		if confidence > 0.5:
# # 			# compute the (x, y)-coordinates of the bounding box for
# # 			# the object
# # 			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
# # 			(startX, startY, endX, endY) = box.astype("int")

# # 			# ensure the bounding boxes fall within the dimensions of
# # 			# the frame
# # 			(startX, startY) = (max(0, startX), max(0, startY))
# # 			(endX, endY) = (min(w - 1, endX), min(h - 1, endY))

# # 			# extract the face ROI, convert it from BGR to RGB channel
# # 			# ordering, resize it to 224x224, and preprocess it
# # 			face = frame[startY:endY, startX:endX]
# # 			face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
# # 			face = cv2.resize(face, (224, 224))
# # 			face = img_to_array(face)
# # 			face = preprocess_input(face)

# # 			# add the face and bounding boxes to their respective
# # 			# lists
# # 			faces.append(face)
# # 			locs.append((startX, startY, endX, endY))

# 	# only make a predictions if at least one face was detected
# 	if len(faces) > 0:
# 		# for faster inference we'll make batch predictions on *all*
# 		# faces at the same time rather than one-by-one predictions
# 		# in the above `for` loop
# 		faces = np.array(faces, dtype="float32")
# 		preds = maskNet.predict(faces, batch_size=32)

# 	# return a 2-tuple of the face locations and their corresponding
# 	# locations
# 	return (locs, preds)

# # load our serialized face detector model from disk
# prototxtPath = r"face_detector\deploy.prototxt"
# weightsPath = r"face_detector\res10_300x300_ssd_iter_140000.caffemodel"
# faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

# # load the face mask detector model from disk
# maskNet = load_model("mask_detector.model")


# # loop over the frames from the video stream
# while True:
# 	# grab the frame from the threaded video stream and resize it
# 	# to have a maximum width of 400 pixels
# 	frame = vs.read()
# 	frame = imutils.resize(frame, width=400)

# 	# detect faces in the frame and determine if they are wearing a
# 	# face mask or not
# 	(locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

# 	# loop over the detected face locations and their corresponding
# 	# locations
# 	for (box, pred) in zip(locs, preds):
# 		# unpack the bounding box and predictions
# 		(startX, startY, endX, endY) = box
# 		(mask, withoutMask) = pred

# 		# determine the class label and color we'll use to draw
# 		# the bounding box and text
# 		label = "Mask" if mask > withoutMask else "No Mask"
# 		color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

# 		# include the probability in the label
# 		label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

# 		# display the label and bounding box rectangle on the output
# 		# frame
# 		cv2.putText(frame, label, (startX, startY - 10),
# 			cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
# 		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
